In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

In [2]:
# Load the dataset from CSV
df = pd.read_csv('flights_data.csv')

# View the first few rows of the dataset
print(df.tail())

       flight_number call_sign   status  is_cargo        airport_name  \
134994       DL 1524   DAL1524  Arrived     False         Saint Louis   
134995       DL 1184   DAL1184  Arrived     False       Oklahoma City   
134996       DL 1062   DAL1062  Arrived     False         Minneapolis   
134997       DL 4940   EDV4940  Arrived     False              Monroe   
134998       DL 5383   EDV5383  Arrived     False  Bloomington/Normal   

       airport_iata airport_icao      scheduled_utc         scheduled_local  \
134994          STL         KSTL  2024-10-17 12:39Z  2024-10-17 08:39-04:00   
134995          OKC         KOKC  2024-10-17 12:28Z  2024-10-17 08:28-04:00   
134996          MSP         KMSP  2024-10-17 12:44Z  2024-10-17 08:44-04:00   
134997          MLU         KMLU  2024-10-17 12:44Z  2024-10-17 08:44-04:00   
134998          BMI         KBMI  2024-10-17 12:52Z  2024-10-17 08:52-04:00   

              revised_utc           revised_local         runway_utc  \
134994  2024-1

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134999 entries, 0 to 134998
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   flight_number    134999 non-null  object
 1   call_sign        127693 non-null  object
 2   status           134999 non-null  object
 3   is_cargo         134999 non-null  bool  
 4   airport_name     134999 non-null  object
 5   airport_iata     131811 non-null  object
 6   airport_icao     131811 non-null  object
 7   scheduled_utc    134999 non-null  object
 8   scheduled_local  134999 non-null  object
 9   revised_utc      125944 non-null  object
 10  revised_local    125944 non-null  object
 11  runway_utc       122955 non-null  object
 12  runway_local     122955 non-null  object
 13  terminal         124168 non-null  object
 14  quality          134999 non-null  object
 15  aircraft_reg     126665 non-null  object
 16  aircraft_model   134980 non-null  object
 17  airline_na

In [4]:
#Drop redundant features
df.drop(columns=['call_sign', 'airport_name', 'airport_icao', 'scheduled_local', 'revised_local', 'runway_local', 'airline_name', 'airline_icao'], inplace=True)
df.dropna(inplace=True)
df

,flight_number,status,is_cargo,airport_iata,scheduled_utc,revised_utc,runway_utc,terminal,quality,aircraft_reg,aircraft_model,airline_iata
0,DL 414,Arrived,False,DFW,2024-06-22 20:53Z,2024-06-22 20:30Z,2024-06-22 20:30Z,S,"Basic, Live",N305DN,Airbus A321,DL
1,DL 3634,Arrived,False,LFT,2024-06-22 20:53Z,2024-06-22 20:31Z,2024-06-22 20:31Z,S,"Basic, Live",N834SK,Canadair CRJ 900,DL
2,DL 2200,Arrived,False,CMH,2024-06-22 20:55Z,2024-06-22 20:31Z,2024-06-22 20:31Z,S,"Basic, Live",N852DN,Boeing 737,DL
3,DL 3030,Arrived,False,GNV,2024-06-22 20:51Z,2024-06-22 20:32Z,2024-06-22 20:32Z,S,"Basic, Live",N893AT,Boeing 717,DL
4,SK 929,Arrived,False,CPH,2024-06-22 21:10Z,2024-06-22 20:33Z,2024-06-22 20:33Z,I,"Basic, Live",LN-RKR,Airbus A330,SK
...,...,...,...,...,...,...,...,...,...,...,...,...
134994,DL 1524,Arrived,False,STL,2024-10-17 12:39Z,2024-10-17 12:25Z,2024-10-17 12:25Z,S,"Basic, Live",N900DU,Boeing 737,DL
134995,DL 1184,Arrived,False,OKC,2024-10-17 12:28Z,2024-10-17 12:26Z,2024-10-17 12:26Z,S,"Basic, Live",N917DU,Boeing 737,DL
134996,DL 1062,Arrived,False,MSP,2024-10-17 12:44Z,2024-10-17 12:27Z,2024-10-17 12:27Z,S,"Basic, Live",N583NW,Boeing 757-300,DL
134997,DL 4940,Arrived,False,MLU,2024-10-17 12:44Z,2024-10-17 12:28Z,2024-10-17 12:28Z,S,"Basic, Live",N928XJ,Canadair CRJ 900,DL


In [5]:
# Load CSV data
df_wea = pd.read_csv('open-meteo-data.csv')
df_wea['time'] = pd.to_datetime(df_wea['time'])  # or df_wea['timestamp']

df_wea.rename(columns={'temperature_2m (°C)': 'temperature_2m'}, inplace=True)
df_wea.rename(columns={'snowfall (cm)': 'snowfall'}, inplace=True)
df_wea.rename(columns={'snow_depth (m)': 'snow_depth'}, inplace=True)
df_wea.rename(columns={'cloud_cover (%)': 'cloud_cover'}, inplace=True)
df_wea.rename(columns={'wind_speed_10m (km/h)': 'wind_speed_10m'}, inplace=True)
df_wea.rename(columns={'wind_direction_10m (°)': 'wind_direction_10m'}, inplace=True)

df_wea

,time,temperature_2m,snowfall,snow_depth,cloud_cover,wind_speed_10m,wind_direction_10m
0,2024-06-22 00:00:00,31.1,0.0,0.0,1,13.5,115
1,2024-06-22 01:00:00,28.2,0.0,0.0,0,8.9,111
2,2024-06-22 02:00:00,26.7,0.0,0.0,0,9.9,109
3,2024-06-22 03:00:00,25.7,0.0,0.0,0,9.4,92
4,2024-06-22 04:00:00,24.7,0.0,0.0,6,7.9,93
...,...,...,...,...,...,...,...
2827,2024-10-17 19:00:00,15.7,0.0,0.0,0,5.8,5
2828,2024-10-17 20:00:00,16.0,0.0,0.0,0,6.7,357
2829,2024-10-17 21:00:00,16.0,0.0,0.0,0,7.4,360
2830,2024-10-17 22:00:00,15.5,0.0,0.0,0,6.1,5


In [6]:
import pandas as pd
import pytz  # For timezone conversions

# Target variable
# Ensure timestamps are parsed correctly (fixing string issue)
df['scheduled_utc'] = pd.to_datetime(df['scheduled_utc'], errors='coerce')
df['runway_utc'] = pd.to_datetime(df['runway_utc'], errors='coerce')

# Check for any nulls after conversion (to catch failed conversions)
if df['scheduled_utc'].isna().any() or df['runway_utc'].isna().any():
    print("Warning: Some datetime conversions failed. Check for invalid formats.")

# Convert 'timestamp' columns to datetime format and set time zones
df['delay'] = (df['runway_utc'] - df['scheduled_utc']).dt.total_seconds()
# Handle the OpenMeteo data (df_wea)
df_wea['timestamp'] = pd.to_datetime(df_wea['time'])  # OpenMeteo data has no timezone initially
# Localize 'df_wea' to 'GMT' and then convert to 'UTC'
if df_wea['timestamp'].dt.tz is None:
    df_wea['timestamp'] = df_wea['timestamp'].dt.tz_localize('GMT')  # Localize to GMT
else:
    df_wea['timestamp'] = df_wea['timestamp'].dt.tz_convert('GMT')  # Convert to GMT if already timezone-aware

df_wea['timestamp'] = df_wea['timestamp'].dt.tz_convert('UTC')  # Convert GMT to UTC

# Handle the df DataFrame (your local data)
if df['scheduled_utc'].dt.tz is None:
    df['timestamp'] = pd.to_datetime(df['scheduled_utc']).dt.tz_localize('UTC')  # Localize to UTC
else:
    df['timestamp'] = pd.to_datetime(df['scheduled_utc']).dt.tz_convert('UTC')  # Convert to UTC if already timezone-aware

# Round timestamps to the nearest hour
df['hour'] = df['timestamp'].dt.floor('H')  # Floors to nearest hour (UTC)
df_wea['hour'] = df_wea['timestamp'].dt.floor('H')  # Floors to nearest hour (UTC)

# Merge DataFrames on the 'hour' column
merged_df = pd.merge(df, df_wea, on='hour', how='inner')

# Print the merged DataFrame
print(merged_df)


       flight_number   status  is_cargo airport_iata  \
0             DL 414  Arrived     False          DFW   
1            DL 3634  Arrived     False          LFT   
2            DL 2200  Arrived     False          CMH   
3            DL 3030  Arrived     False          GNV   
4            WN 1429  Arrived     False          JAN   
...              ...      ...       ...          ...   
112076       DL 1184  Arrived     False          OKC   
112077       DL 1062  Arrived     False          MSP   
112078       DL 4940  Arrived     False          MLU   
112079       DL 5383  Arrived     False          BMI   
112080       DL 1108  Arrived     False          SYR   

                   scheduled_utc        revised_utc                runway_utc  \
0      2024-06-22 20:53:00+00:00  2024-06-22 20:30Z 2024-06-22 20:30:00+00:00   
1      2024-06-22 20:53:00+00:00  2024-06-22 20:31Z 2024-06-22 20:31:00+00:00   
2      2024-06-22 20:55:00+00:00  2024-06-22 20:31Z 2024-06-22 20:31:00+00:00   
3  

In [7]:
df=merged_df

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112081 entries, 0 to 112080
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   flight_number       112081 non-null  object             
 1   status              112081 non-null  object             
 2   is_cargo            112081 non-null  bool               
 3   airport_iata        112081 non-null  object             
 4   scheduled_utc       112081 non-null  datetime64[ns, UTC]
 5   revised_utc         112081 non-null  object             
 6   runway_utc          112081 non-null  datetime64[ns, UTC]
 7   terminal            112081 non-null  object             
 8   quality             112081 non-null  object             
 9   aircraft_reg        112081 non-null  object             
 10  aircraft_model      112081 non-null  object             
 11  airline_iata        112081 non-null  object             
 12  delay           

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Assuming 'df' is your DataFrame

# Step 1: Encode categorical variables
label_cols = [ 'status', 'airport_iata', 'terminal', 'quality', 'aircraft_reg', 
              'aircraft_model', 'airline_iata']

label_encoder = LabelEncoder()

# Encoding each column in label_cols
for col in label_cols:
    df[col] = label_encoder.fit_transform(df[col])

# Step 2: Handle missing values
df['snow_depth'].fillna(0, inplace=True)

# Step 3: Convert datetime columns to numeric values (e.g., UNIX timestamp)
df['scheduled_utc_numeric'] = df['scheduled_utc'].view('int64') / 10**9  # UNIX timestamp in seconds
df['runway_utc_numeric'] = df['runway_utc'].view('int64') / 10**9  # UNIX timestamp in seconds

# If 'revised_utc' is useful, convert it to a numeric format, e.g., UNIX timestamp
# If 'revised_utc' is not needed, you can skip this step or drop the column
df['revised_utc_numeric'] = pd.to_datetime(df['revised_utc'], errors='coerce').view('int64') / 10**9

# Step 4: Extract additional time-related features (hour, day of the week, month)
df['hour'] = df['scheduled_utc'].dt.hour
df['day_of_week'] = df['scheduled_utc'].dt.dayofweek
df['month'] = df['scheduled_utc'].dt.month

# Step 5: Drop columns that are not needed for prediction
df = df.drop(['flight_number', 'timestamp_x', 'timestamp_y', 'time', 'scheduled_utc', 'runway_utc', 'revised_utc'], axis=1)

# Step 6: Define the feature matrix X and target variable y
X = df.drop('delay', axis=1)  # Features
y = df['delay']  # Target variable (delay)

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Initialize the XGBoost regressor model
xg_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                      max_depth=5, alpha=10, n_estimators=100)

# Step 9: Train the model
xg_reg.fit(X_train, y_train)

# Step 10: Make predictions on the test set
y_pred = xg_reg.predict(X_test)

# Step 11: Evaluate the model performance
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Step 12: Optionally, print feature importances
importances = xg_reg.feature_importances_
feature_importance_df = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Print feature importances
print(feature_importance_df)


Mean Squared Error: 7872451.217905133
                  feature  importance
16     runway_utc_numeric    0.182037
17    revised_utc_numeric    0.128924
3                terminal    0.127016
15  scheduled_utc_numeric    0.093115
18            day_of_week    0.087499
19                  month    0.074795
7            airline_iata    0.067309
8                    hour    0.039877
12            cloud_cover    0.039579
13         wind_speed_10m    0.033283
9          temperature_2m    0.032853
14     wind_direction_10m    0.027722
6          aircraft_model    0.024205
2            airport_iata    0.022206
5            aircraft_reg    0.019578
0                  status    0.000000
11             snow_depth    0.000000
1                is_cargo    0.000000
4                 quality    0.000000
10               snowfall    0.000000


In [10]:
from sklearn.metrics import mean_squared_error, r2_score
r2 = r2_score(y_test, y_pred)

In [11]:
res_dict = { 
        "MSE": str(mse),
        "R squared": str(r2),
    }

In [12]:
df

,status,is_cargo,airport_iata,terminal,quality,aircraft_reg,aircraft_model,airline_iata,delay,hour,...,snowfall,snow_depth,cloud_cover,wind_speed_10m,wind_direction_10m,scheduled_utc_numeric,runway_utc_numeric,revised_utc_numeric,day_of_week,month
0,0,False,61,2,0,998,6,9,-1380.0,20,...,0.0,0.0,76,11.8,130,1.719090e+09,1.719088e+09,1.719088e+09,5,6
1,0,False,122,2,0,3311,51,9,-1320.0,20,...,0.0,0.0,76,11.8,130,1.719090e+09,1.719088e+09,1.719088e+09,5,6
2,0,False,46,2,0,3406,25,9,-1440.0,20,...,0.0,0.0,76,11.8,130,1.719090e+09,1.719088e+09,1.719088e+09,5,6
3,0,False,88,2,0,3958,21,9,-1140.0,20,...,0.0,0.0,76,11.8,130,1.719089e+09,1.719088e+09,1.719088e+09,5,6
4,0,False,114,1,0,3020,28,21,0.0,20,...,0.0,0.0,76,11.8,130,1.719088e+09,1.719088e+09,1.719088e+09,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112076,0,False,162,2,0,4111,25,9,-120.0,12,...,0.0,0.0,0,5.8,353,1.729168e+09,1.729168e+09,1.729168e+09,3,10
112077,0,False,151,2,0,2289,37,9,-1020.0,12,...,0.0,0.0,0,5.8,353,1.729169e+09,1.729168e+09,1.729168e+09,3,10
112078,0,False,148,2,0,4201,51,9,-960.0,12,...,0.0,0.0,0,5.8,353,1.729169e+09,1.729168e+09,1.729168e+09,3,10
112079,0,False,24,2,0,959,51,9,-1440.0,12,...,0.0,0.0,0,5.8,353,1.729170e+09,1.729168e+09,1.729168e+09,3,10


In [13]:
import hopsworks
import os

project = hopsworks.login()
fs = project.get_feature_store()

2025-01-10 13:54:17,578 INFO: Initializing external client
2025-01-10 13:54:17,579 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-10 13:54:19,617 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1162390


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112081 entries, 0 to 112080
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   status                 112081 non-null  int32  
 1   is_cargo               112081 non-null  bool   
 2   airport_iata           112081 non-null  int32  
 3   terminal               112081 non-null  int32  
 4   quality                112081 non-null  int32  
 5   aircraft_reg           112081 non-null  int32  
 6   aircraft_model         112081 non-null  int32  
 7   airline_iata           112081 non-null  int32  
 8   delay                  112081 non-null  float64
 9   hour                   112081 non-null  int32  
 10  temperature_2m         112081 non-null  float64
 11  snowfall               112081 non-null  float64
 12  snow_depth             112081 non-null  float64
 13  cloud_cover            112081 non-null  int64  
 14  wind_speed_10m         112081 non-nu

In [15]:
import joblib

# Save the XGBoost model to a file
model_filename = "xgboost_flight_delay_model.pkl"
joblib.dump(xg_reg, model_filename)


['xgboost_flight_delay_model.pkl']

In [17]:
mr = project.get_model_registry()


model = mr.python.create_model(
    name="xgboost_flight_delay_model", 
    metrics= res_dict,
)

In [18]:
# Save the model metadata to Hopsworks Model Registry
model.save(model_filename)


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/273144 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1162390/models/xgboost_flight_delay_model/1


Model(name: 'xgboost_flight_delay_model', version: 1)